In [43]:
import numpy as np
import pandas as pd
from model import RegularizedLogisticRegression

In [44]:
df = pd.read_csv('./data/insurance.csv')

# sex
sex_dummies = pd.get_dummies(df['sex'])
df = pd.concat([df, sex_dummies], axis=1)
df = df.drop(['sex'], axis=1)

# smoker
smoker_dummies = pd.get_dummies(df['smoker'])
df = pd.concat([df, smoker_dummies], axis=1)
df = df.drop(['smoker'], axis=1)

# region
region_dummies = pd.get_dummies(df['region'])
df = pd.concat([df, region_dummies], axis=1)
df = df.drop(['region'], axis=1)

df = df.drop(columns=df.columns[0])
df

,bmi,children,charges,female,male,no,yes,northeast,northwest,southeast,southwest
0,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [45]:
df = df.sample(frac=1).reset_index(drop=True)
df

,bmi,children,charges,female,male,no,yes,northeast,northwest,southeast,southwest
0,33.440,1,8334.58960,0,1,1,0,1,0,0,0
1,27.360,3,8606.21740,0,1,1,0,1,0,0,0
2,27.645,0,10594.50155,0,1,1,0,0,1,0,0
3,20.900,1,1832.09400,0,1,1,0,0,0,0,1
4,31.100,0,1526.31200,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1333,27.700,0,3554.20300,1,0,1,0,0,0,0,1
1334,35.720,0,2404.73380,1,0,1,0,0,1,0,0
1335,25.300,2,24667.41900,1,0,0,1,0,0,1,0
1336,34.485,3,60021.39897,0,1,0,1,0,1,0,0


In [46]:
X = df.drop(['charges'], axis=1)
X

,bmi,children,female,male,no,yes,northeast,northwest,southeast,southwest
0,33.440,1,0,1,1,0,1,0,0,0
1,27.360,3,0,1,1,0,1,0,0,0
2,27.645,0,0,1,1,0,0,1,0,0
3,20.900,1,0,1,1,0,0,0,0,1
4,31.100,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1333,27.700,0,1,0,1,0,0,0,0,1
1334,35.720,0,1,0,1,0,0,1,0,0
1335,25.300,2,1,0,0,1,0,0,1,0
1336,34.485,3,0,1,0,1,0,1,0,0


In [47]:
y = df[['charges']]
y

,charges
0,8334.58960
1,8606.21740
2,10594.50155
3,1832.09400
4,1526.31200
...,...
1333,3554.20300
1334,2404.73380
1335,24667.41900
1336,60021.39897


In [48]:
X.to_csv('./data/preprocessed_X.csv', index=False)
y.to_csv('./data/preprocessed_y.csv', index=False)

In [49]:
X = open('./data/preprocessed_X.csv', 'r').readlines()
y = open('./data/preprocessed_y.csv', 'r').readlines()

In [50]:
filename = 0

# reserve 20% for testing, split the rest into 4 parts
num_rows = len(X)
step = (num_rows - 1) // 5
for i in range(1, num_rows, step):
    if i + step > num_rows:
        break
    if filename == 4:
        open('./data/test_X.csv', 'w+').writelines(X[i:i+step])
        open('./data/test_y.csv', 'w+').writelines(y[i:i+step])
    else:
        open('./data/train_X_' + str(filename) + '.csv', 'w+').writelines(X[i:i+step])
        open('./data/train_y_' + str(filename) + '.csv', 'w+').writelines(y[i:i+step])
    filename += 1

In [51]:
a = pd.read_csv('./data/train_X_0.csv', header=None, index_col=None)
a

,0,1,2,3,4,5,6,7,8,9
0,33.440,1,0,1,1,0,1,0,0,0
1,27.360,3,0,1,1,0,1,0,0,0
2,27.645,0,0,1,1,0,0,1,0,0
3,20.900,1,0,1,1,0,0,0,0,1
4,31.100,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
262,32.300,2,0,1,1,0,0,0,0,1
263,25.080,0,0,1,1,0,0,0,1,0
264,26.900,0,0,1,1,0,0,0,0,1
265,33.400,0,1,0,1,0,0,0,0,1
